In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
pip install -U segmentation-models

In [3]:
def extract_data(folderpath):
    """ (ETH) Extract the images into a 4D tensor [image index, y, x, channels].
    Values are rescaled from [0, 255] down to [-0.5, 0.5].

    Parameters
    ----------
    filename: string
        The name of the image file
    num_images: int
        The number of images that should be extracted

    Returns
    -------
    data: ndarray
        A numpy array containting the images
    """
    files = os.listdir(folderpath)
    n = len(files)
    imgs = [cv2.resize(mpimg.imread(folderpath+files[i]),(384,384)) for i in range(n)]
    data = np.asarray(imgs)
    return data

def extract_data_test(folderpath):
    """ (ETH) Extract the images into a 4D tensor [image index, y, x, channels].
    Values are rescaled from [0, 255] down to [-0.5, 0.5].

    Parameters
    ----------
    filename: string
        The name of the image file
    num_images: int
        The number of images that should be extracted

    Returns
    -------
    data: ndarray
        A numpy array containting the images
    """
    imgs=[]
    for i in range(1,51):
      img = mpimg.imread(folderpath+'test_%d.png'%i)
      img = cv2.resize(img, (384,384))
      imgs.append(img)
    data = np.asarray(imgs)
    return data

def value_to_class(v):
    ''' (ETH) Assign labels to a patch v
    
    Parameters
    ----------
    v: ndarray
        The patch
    
    Returns
    --------
    [0,1] if road, [1,0] elsewise
        Labels for the patch'''

    foreground_threshold = 0.25  # percentage of pixels > 1 required to assign a foreground label to a patch
    df = np.sum(v)
    if df > foreground_threshold:  # road
        return [1,0]
    else:  # bgrd
        return [0,1]


def extract_labels(folderpath):
    """ (ETH) Extract the labels into a 1-hot matrix [image index, label index].
    
    Parameters
    ----------
    filename: string
        The name of the image file
    num_images: int
        The number of images
    
    Returns
    --------
    labels: ndarray
        1-hot matrix [image index, label index]
    """
    gt_imgs = []
    files = os.listdir(folderpath)
    n = len(files)
    for i in range(n):
        img = mpimg.imread(folderpath+files[i])
        img = cv2.resize(img, (384,384))
        try:
            gt_imgs.append(img[:,:,0])
        except:
            gt_imgs.append(img)

    #labels = np.asarray([value_to_class(gt_imgs[i]) for i in range(len(gt_imgs))])
    # Convert to dense 1-hot representation.
    #labels = labels.astype(np.float32)
    return np.asarray(gt_imgs)

In [4]:
!unzip "/content/drive/My Drive/ml/training.zip" -d "/content"

Archive:  /content/drive/My Drive/ml/training.zip
replace /content/training/groundtruth/satImage_001.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [8]:
import os
import matplotlib.image as mpimg
from PIL import Image
import cv2

from sklearn.model_selection import train_test_split

train_data_path = '/content/training/images/'
train_labels_path = '/content/training/groundtruth/'

train_data = extract_data(train_data_path)
train_labels = extract_labels(train_labels_path)

In [9]:
train_data.shape

(1700, 384, 384, 3)

In [10]:
train_labels.shape

(1700, 384, 384)

In [11]:
import segmentation_models as sm
sm.set_framework('tf.keras')
import math
keras.backend.clear_session()
BACKBONE = 'resnet50'
preprocess_input = sm.get_preprocessing(BACKBONE)
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(train_data, train_labels, test_size=0.2, random_state=1)


# preprocess input
x_train = preprocess_input(x_train)
#y_train = preprocess_input(y_train)

print(x_train.shape)
print(y_train.shape)

Segmentation Models: using `keras` framework.
(1700, 384, 384, 3)
(1700, 384, 384)


In [ ]:

# define model
model = sm.Unet(BACKBONE, encoder_weights='imagenet')
model.compile(
    'Adam',
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score, sm.metrics.FScore(),'accuracy'],
)
callbacks = [keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/Models/Unet_rike.h5", save_best_only=True)]
# fit model
# if you use data generator use model.fit_generator(...) instead of model.fit(...)
# more about `fit_generator` here: https://keras.io/models/sequential/#fit_generator
model.fit(x=x_train, y=y_train,
   epochs=100, batch_size=8,
   callbacks=callbacks,
   validation_data=(x_val, y_val)
)


Epoch 1/100
170/170 [==============================] - ETA: 0s - loss: 0.9962 - iou_score: 0.3568 - f1-score: 0.5179 - accuracy: 0.6990

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


170/170 [==============================] - 205s 1s/step - loss: 0.9962 - iou_score: 0.3568 - f1-score: 0.5179 - accuracy: 0.6990 - val_loss: 1.7608 - val_iou_score: 0.0198 - val_f1-score: 0.0388 - val_accuracy: 0.7518
Epoch 2/100
170/170 [==============================] - 175s 1s/step - loss: 0.7936 - iou_score: 0.4895 - f1-score: 0.6558 - accuracy: 0.7216 - val_loss: 1.8833 - val_iou_score: 0.0129 - val_f1-score: 0.0254 - val_accuracy: 0.7518
Epoch 3/100
170/170 [==============================] - 175s 1s/step - loss: 0.7251 - iou_score: 0.5355 - f1-score: 0.6957 - accuracy: 0.7261 - val_loss: 1.8197 - val_iou_score: 0.0134 - val_f1-score: 0.0265 - val_accuracy: 0.7518
Epoch 4/100
170/170 [==============================] - 177s 1s/step - loss: 0.6589 - iou_score: 0.5768 - f1-score: 0.7302 - accuracy: 0.7322 - val_loss: 1.5478 - val_iou_score: 0.0443 - val_f1-score: 0.0848 - val_accuracy: 0.7518
Epoch 5/100
170/170 [==============================] - 177s 1s/step - loss: 0.6302 - iou_sco

In [11]:
!unzip "/content/drive/My Drive/ml/testing.zip" -d "/content"

Archive:  /content/drive/My Drive/ml/testing.zip
  inflating: /content/testing/test_1.png  
  inflating: /content/testing/test_10.png  
  inflating: /content/testing/test_11.png  
  inflating: /content/testing/test_12.png  
  inflating: /content/testing/test_13.png  
  inflating: /content/testing/test_14.png  
  inflating: /content/testing/test_15.png  
  inflating: /content/testing/test_16.png  
  inflating: /content/testing/test_17.png  
  inflating: /content/testing/test_18.png  
  inflating: /content/testing/test_19.png  
  inflating: /content/testing/test_2.png  
  inflating: /content/testing/test_20.png  
  inflating: /content/testing/test_21.png  
  inflating: /content/testing/test_22.png  
  inflating: /content/testing/test_23.png  
  inflating: /content/testing/test_24.png  
  inflating: /content/testing/test_25.png  
  inflating: /content/testing/test_26.png  
  inflating: /content/testing/test_27.png  
  inflating: /content/testing/test_28.png  
  inflating: /content/testing

In [64]:
from keras.models import load_model
model = load_model('/content/drive/MyDrive/Models/Unet_rike.h5', custom_objects = {'binary_crossentropy_plus_jaccard_loss':sm.losses.bce_jaccard_loss, 'iou_score': sm.metrics.iou_score, 'f1-score': sm.metrics.FScore()})

test_images = extract_data_test('/content/testing/')



In [65]:
print(test_images[0].shape)

(384, 384, 3)


In [66]:
result = model.predict(preprocess_input(test_images))



In [67]:
def img_float_to_uint8(img):
    '''converts image array with floats to uint8
    
    parameters
    -----------
    img: ndarray
        image array
    
    returns
    -------
    rimg: ndarray
        converted array'''

    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg


for i in range(len(result)):
  img = cv2.resize(result[i],(608,608))
  w = img.shape[0]
  h = img.shape[1]
  gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
  gt_img8 = img_float_to_uint8(img)          
  gt_img_3c[:, :, 0] = gt_img8
  gt_img_3c[:, :, 1] = gt_img8
  gt_img_3c[:, :, 2] = gt_img8

  cv2.imwrite('/content/Predictions/test%d.png'%(i+1),gt_img_3c)


In [68]:
! python mask_to_submission.py

/content/Predictions/test1.png
/content/Predictions/test2.png
/content/Predictions/test3.png
/content/Predictions/test4.png
/content/Predictions/test5.png
/content/Predictions/test6.png
/content/Predictions/test7.png
/content/Predictions/test8.png
/content/Predictions/test9.png
/content/Predictions/test10.png
/content/Predictions/test11.png
/content/Predictions/test12.png
/content/Predictions/test13.png
/content/Predictions/test14.png
/content/Predictions/test15.png
/content/Predictions/test16.png
/content/Predictions/test17.png
/content/Predictions/test18.png
/content/Predictions/test19.png
/content/Predictions/test20.png
/content/Predictions/test21.png
/content/Predictions/test22.png
/content/Predictions/test23.png
/content/Predictions/test24.png
/content/Predictions/test25.png
/content/Predictions/test26.png
/content/Predictions/test27.png
/content/Predictions/test28.png
/content/Predictions/test29.png
/content/Predictions/test30.png
/content/Predictions/test31.png
/content/Predicti